#### Install and libraries

In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q apache_beam
# !pip install git+https://github.com/huggingface/transformers.git
!pip install -q huggingface-hub
import os
import re
import json
from datasets import Dataset
import datasets
from collections import Counter
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import transformers
import pyarrow as pa
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, TFAutoModel
import json
from datasets import Dataset
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K

     |████████████████████████████████| 4.4 MB 7.5 MB/s 
     |████████████████████████████████| 101 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 35.7 MB/s 
     |████████████████████████████████| 6.6 MB 39.7 MB/s 
     |████████████████████████████████| 362 kB 6.9 MB/s 
     |████████████████████████████████| 1.1 MB 50.9 MB/s 
     |████████████████████████████████| 140 kB 45.0 MB/s 
     |████████████████████████████████| 212 kB 12.1 MB/s 
     |████████████████████████████████| 127 kB 5.0 MB/s 
     |████████████████████████████████| 94 kB 1.4 MB/s 
     |████████████████████████████████| 271 kB 13.8 MB/s 
     |████████████████████████████████| 144 kB 47.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 10.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Functions

### 1. Load JSON

In [ ]:
def load_json(path):
  with open(path, 'r') as json_file:
      with open(path, 'r') as j:
        contents = json.loads(j.read())
      # ('json', data_files='drive/MyDrive/w266/data/natural_questions/data_chunks/761.json')
        df = pd.DataFrame(contents)
        data = Dataset.from_pandas(df)
  return data

### 2. function to add start_byte key to answers.

In [ ]:
def start_btye(examples):
  res = [(ele.start(), ele.end()) for ele in re.finditer(r'\S+', examples['context'])]
  answer_start = examples['answers']['answer_start']
  output = []
  if len(answer_start) != 0:
    for i in answer_start:
      byte_index = res[i][0]
      output.append(byte_index)

  examples['answers']['start_byte'] = output
  return examples

### 3. preprocess_training_examples

In [ ]:
def preprocess_training_examples(examples):
  questions = [q.strip() for q in examples["question"]]
  inputs = tokenizer(
  questions,
  examples["context"],
  max_length=max_length,
  truncation="only_second",
  stride=stride,
  return_overflowing_tokens=True,
  return_offsets_mapping=True,
  padding='max_length'
  )

  offset_mapping = inputs.pop("offset_mapping")
  sample_map = inputs.pop("overflow_to_sample_mapping")
  # offset_mapping = inputs["offset_mapping"]
  # sample_map = inputs["overflow_to_sample_mapping"]
  answers = examples["answers"]
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):   # i is th of generated feature

    sample_idx = sample_map[i]   # sample id in data
    answer = answers[sample_idx]  # original answer dictionary
    
    # Find the start and end of the context
    sequence_ids = inputs.sequence_ids(i)
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    num_answers = len(answer['start_byte'])  # num of answers in each sample
    start =[]
    end = []
    if num_answers == 0:
      pass
    elif num_answers != 0:
      for j in range(num_answers):
        start_char = answer['start_byte'][j]
        end_char = answer['start_byte'][j] + len(answer["text"][j])

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start.append(0)
            end.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start.append(idx - 1)
            # start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end.append(idx + 1)
            # end_positions.append(idx + 1)

    start_positions.append(start)
    end_positions.append(end)
  
  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  
  return inputs

## train - Generator -not done 7/7

In [ ]:
from transformers import BertTokenizer, TFBertModel   #AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained('bert-base-cased')
bert_model.trainable = False

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
type(bert_model)

transformers.models.bert.modeling_tf_bert.TFBertModel

In [ ]:
# import datasets
train_path = 'drive/MyDrive/w266/data/natural_questions/data_chunks/761.json'
train = load_json(train_path)
train_parsed = train.map(start_btye)

max_length = 300
stride = 50
train_dataset = train_parsed.map(
    preprocess_training_examples,
    batched=True, remove_columns=train_parsed.column_names,
    )
len(train_parsed), len(train_dataset)

  0%|          | 0/148 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

(148, 387)

In [ ]:
len(train_dataset['input_ids'][0])

300

In [ ]:
val_path = 'drive/MyDrive/w266/data/natural_questions/data_chunks/1.json'
val = load_json(val_path)
val_parsed = val.map(start_btye)

val_dataset = val_parsed.map(
    preprocess_training_examples,
    batched=True, remove_columns=val_parsed.column_names,
    )
len(val_parsed), len(val_dataset)

  0%|          | 0/200 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

(200, 636)

### Comment：tokenzier output
The 4 examples gave 27 features, 27 lists (each has 100 tokens, [SEP] <question> [CLS] <context> [SEP] <pad>) inputs['input_ids']. each token is a number, correponding to the tokenizer model. 

inputs['token_type_ids'] has 27 list items, 0 - question, 1 - context, 0- pad. 

inputs['attention_mask'] has 27 list items, 1 - existed word from question or context, 0- pad. 


inputs['offset_mappting']: list of 27 lists, each feature list contain tuples of two integers representing the span of charcters(letters and space...) in the context. 

inputs['overflow_to_sample_mapping']: just a list of integers. tells example/sample the current feature is coming from. 

Model functions:


In [ ]:
def get_shape_list(tensor, expected_rank=None, name=None):
  """Returns a list of the shape of tensor, preferring static dimensions.
  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.
  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
  if name is None:
    name = tensor.name

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape

## MODELS

## Function to prep training data features and labels

In [ ]:
def prep_features(dataset, features=['input_ids', 'token_type_ids', 'attention_mask']):

  tf_data = dataset.with_format("tensorflow")
  tf_features = [tf_data[x] for x in features if x != "example_id"]
  tf_features = tf.reshape(tf.stack(tf_features),(len(dataset), 300, 3))
  return tf_features

def prep_labels(dataset, labels=['start_positions', 'end_positions']):
  tf_data = dataset.with_format("tensorflow")
  start_positions = tf_data[labels[0]].to_tensor()
  end_positions = tf_data[labels[1]].to_tensor()
  tf_labels = tf.reshape(tf.stack([start_positions, end_positions]), (len(dataset), K.shape(start_positions)[1], 2))
  return tf_labels

train_features= prep_features(train_dataset, features=['input_ids', 'token_type_ids', 'attention_mask'])
train_labels = prep_labels(train_dataset, labels=['start_positions', 'end_positions'])


### Total Loss function

In [ ]:
def total_loss(y_true, y_pred):

  start_logits, end_logits = y_pred[:,:,0], y_pred[:,:,1]

  start_positions, end_positions = y_true[:,:,0], y_true[:,:,1]

  def compute_loss(logits, positions, seq_length):
    one_hot_positions = tf.one_hot(
        positions, depth=seq_length, dtype=tf.float32)
    
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    d1, d2 = K.shape(log_probs)[0], K.shape(log_probs)[1]
    t1, t2 = one_hot_positions.shape[0], one_hot_positions.shape[2]
    log_probs = tf.reshape(log_probs, [d1, -1, d2])
    loss = -tf.reduce_mean(
        tf.reduce_sum(one_hot_positions * log_probs, axis=-1))
    return loss
    
  start_loss = compute_loss(start_logits, start_positions, seq_length=300)
  end_loss = compute_loss(end_logits, end_positions,seq_length= 300)
  total_loss = (start_loss + end_loss) / 2.0
  return total_loss

## Baseline Model: Bidirectional LSTM

In [ ]:
batch_size=32
max_length=300
learning_rate=0.0005

model = keras.Sequential()
model.add(layers.Input(shape=(max_length,3), dtype="int32"))  # [None, 300, 3]
model.add(layers.Embedding(input_dim=50000, output_dim=1, name='embeddings')) 
model.add(layers.Reshape((300, 3))) # [None, 300, 3, 128]
model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True, name='BiLSTM')))  # [None, 300, 128]
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True, name = 'BiGRU')))  # [None, 300, 128]
model.add(layers.Dense(2, activation="softmax", name="output_layer")) # [None, 300, 2]

model.compile(loss=total_loss, optimizer=keras.optimizers.Adam(learning_rate=learning_rate))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embeddings (Embedding)      (None, 300, 3, 1)         50000     
                                                                 
 reshape (Reshape)           (None, 300, 3)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 300, 128)         34816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 300, 128)         74496     
 nal)                                                            
                                                                 
 output_layer (Dense)        (None, 300, 2)            258       
                                                                 
Total params: 159,570
Trainable params: 159,570
Non-trai

In [ ]:
model.fit(train_features, train_labels, epochs=5)

Epoch 1/5
13/13 [==============================] - 16s 609ms/step - loss: 5.7038
Epoch 2/5
13/13 [==============================] - 8s 648ms/step - loss: 5.7038
Epoch 3/5
13/13 [==============================] - 13s 1s/step - loss: 5.7038
Epoch 4/5
13/13 [==============================] - 17s 1s/step - loss: 5.7038
Epoch 5/5
13/13 [==============================] - 10s 780ms/step - loss: 5.7038


In [ ]:
predictions = model.predict(eval_features)

In [ ]:
# the prediction is shape (len(tokenized_dataset, max_length, 2))
predictions.shape

### BERT_QA

In [ ]:
def bertQA(batch_size=32, max_length=300, train_layers=-1, learning_rate=0.00005): 
    # use pretained BERTmodel 
    bert_model = TFBertModel.from_pretrained('bert-base-cased')
    
    if not train_layers == -1:
        retrain_layers = []
        for retrain_layer_number in range(train_layers):
            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)
        for w in outputs.weights:
            if not any([x in w.name for x in retrain_layers]):
                w._trainable = False
    
    input_ids = layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
    token_type_ids = layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids')

    inputs = [input_ids, attention_mask, token_type_ids]
    
    bert_output = bert_model(inputs)
    last_hidden_state = bert_output.last_hidden_state  # shape:batch_size, sequence_length, hidden_size
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(last_hidden_state)
    dropout = tf.keras.layers.Dropout(rate=0.1, name='dropout')(dense)

    outputs = layers.Dense(2, activation="softmax")(dropout)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss=total_loss, optimizer=keras.optimizers.Adam(learning_rate=learning_rate))

    model.summary()
    
    return model

In [ ]:
bertQA = bertQA(batch_size=32, max_length=300, train_layers=-1, learning_rate=0.00005)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 300)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 tf_bert_model_5 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',   

In [ ]:
features=['input_ids', 'token_type_ids', 'attention_mask']
dataset=train_dataset
tf_data = dataset.with_format("tensorflow")
tf_features = [tf_data[x] for x in features if x != "example_id"]
tf_features

[<tf.Tensor: shape=(387, 300), dtype=int64, numpy=
 array([[  101,  1150,  1144, ...,  1210,  1149,   102],
        [  101,  1150,  1144, ...,     0,     0,     0],
        [  101,  1184,  1674, ...,     0,     0,     0],
        ...,
        [  101,  1184,  1110, ...,   135,   133,   102],
        [  101,  1184,  1110, ...,   135, 24689,   102],
        [  101,  1184,  1110, ...,     0,     0,     0]])>,
 <tf.Tensor: shape=(387, 300), dtype=int64, numpy=
 array([[0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 0, 0]])>,
 <tf.Tensor: shape=(387, 300), dtype=int64, numpy=
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0]])>]

In [ ]:
bertQA.fit(tf_features, train_labels, epochs=5)

Epoch 1/5
